# YOLOv3를 이용한 실시간 객체 탐지

## 데이터 생성기 만들기고 옵니마이저 지정하기

In [1]:
import tensorflow as tf
from config import *
from data import DataGenerator 

trainset = DataGenerator(data_path=TRAIN_DATA_PATH,
                         annot_path=TRAIN_ANNOT_PATH,
                         class_label_path=CLASS_LABEL_PATH)
testset = DataGenerator(data_path=TEST_DATA_PATH, 
                        annot_path=TEST_ANNOT_PATH,
                        class_label_path=CLASS_LABEL_PATH)
steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64) 
warmup_steps = WARMUP_EPOCHS * steps_per_epoch
total_steps = EPOCHS * steps_per_epoch

optimizer = tf.keras.optimizers.Adam()

## 학습시키기

In [2]:
import os
from config import *
from bbox_iou import bbox_iou, bbox_giou
from yolov3 import Create_YOLOv3
from train import *


yolo = Create_YOLOv3(num_class=NUM_CLASS, train_mode=True)

best_val_loss = 99999
save_directory = os.path.join(CHECKPOINTS_FOLDER, MODEL_NAME)

for epoch in range(EPOCHS):
    for image_data, target in trainset:
        results = train_step(yolo, image_data, target)
        cur_step = results[0] % steps_per_epoch
        print("epoch:{:2.0f} step:{:5.0f}/{}, lr:{:.6f}, giou_loss:{:7.2f}, conf_loss:{:7.2f}, prob_loss:{:7.2f}, total_loss:{:7.2f}".format(epoch, cur_step, steps_per_epoch, results[1], results[2], results[3], results[4], results[5]))
 
    if len(testset) == 0: 
        print("configure TEST options to validate model")
        yolo.save_weights(save_directory)
        continue 

    count = 0
    giou_val, conf_val, prob_val, total_val = 0, 0, 0, 0 

    for image_data, target in testset:
        results = validate_step(yolo, image_data, target)
        count += 1
        giou_val += results[0]
        conf_val += results[1]
        prob_val += results[2]
        total_val += results[3]

    # validation loss 저장 
    with validate_writer.as_default():
        tf.summary.scalar("validate_loss/total_val", total_val / count, step=epoch)
        tf.summary.scalar("validate_loss/giou_val", giou_val / count, step=epoch)
        tf.summary.scalar("validate_loss/conf_val", conf_val / count, step=epoch)
        tf.summary.scalar("validate_loss/prob_val", prob_val / count, step=epoch)
    validate_writer.flush()
    # print("giou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".format( giou_val / count, conf_val / count, prob_val / count, total_val / count))

    if SAVE_CHECKPOINT and not SAVE_BEST_ONLY:
        save_directory = os.path.join(CHECKPOINTS_FOLDER,  MODEL_NAME + "_epoch_{:03d}_val_loss_{:7.2f}".format(epoch, total_val / count))

    # print(best_val_loss, total_val / count)

    if SAVE_BEST_ONLY:
        if best_val_loss > total_val / count:
            best_val_loss = total_val / count
            yolo.save_weights(save_directory)

GPUs []
epoch: 0 step:    2/138, lr:0.000001, giou_loss:  10.48, conf_loss:1789.77, prob_loss:  19.08, total_loss:1819.32
epoch: 0 step:    3/138, lr:0.000001, giou_loss:  11.70, conf_loss:1852.12, prob_loss:  21.80, total_loss:1885.62


KeyboardInterrupt: 

In [ ]:
# yolo.save_weights(save_directory)